# Install Relevant Libraries

In [ ]:
!pip install selenium
!pip install BeautifulSoup4
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# Import Relevant Libraries

In [ ]:
import  numpy as np 
import  matplotlib.pyplot as plt 
import  pandas as pd
import  os
import  re 
from    selenium import webdriver
from    selenium.webdriver.common.keys import Keys
from    selenium.webdriver.support import expected_conditions as EC
from    selenium.webdriver.support.ui import WebDriverWait
from    selenium.webdriver.common.by import By
from    selenium.webdriver.common.action_chains import ActionChains
import  time
import  urllib3
from    bs4 import BeautifulSoup
import  csv
import  requests 
from    urllib import request
from    google.colab import files
import  sys
from    selenium import webdriver
import  json 
from    urllib.request import urlopen

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')
df=pd.read_csv('gdrive/My Drive/Colab Notebooks/FYP/Reuters List.csv')

# Import the File

# Web Scraper

In [ ]:
for index, row in df.iloc[0:1].iterrows():
    sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',options=chrome_options)
    company_name = row['description']
    print(company_name)
    url = row['url']
    
    if not pd.isna(url):
      print(url)
      driver.get(url)
      #wait = WebDriverWait(driver, 40)
      #button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="accept-recommended-btn-handler"]')))
      #driver.execute_script("arguments[0].click()", button)
      
      scrollNumber = 100

      for i in range(0, 100):
      
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # time.sleep(3)
      
      html = driver.page_source
      f = open('{0}.html'.format(company_name), 'w')
      f.write(html.encode('utf-8').decode('ascii', 'ignore'))
      f.close()
      driver.close()
      http=urllib3.PoolManager()
      data = open('{0}.html'.format(company_name),'r')
      
      soup = BeautifulSoup(data, 'html.parser')
      data1 = soup.find('div', attrs = {'class': 'FeedScroll-feed-container-106s7'})
      
      links = []
      for link in data1.find_all('a'): 
        links.append(link.get('href'))
      
      links = list(dict.fromkeys(links))

      locals()["df" + str(company_name)] = pd.DataFrame(columns = ['Date', 'Headline', 'Text'])

      for link in links:
        try:
          url = link
          page = urlopen(url)
          html_bytes = page.read()
          html = html_bytes.decode("utf-8")
          soup = BeautifulSoup(html, 'html.parser')
          try:
              head = soup.find('h1', attrs = {'class': 'Text__text___3eVx1j Text__dark-grey___AS2I_p Text__medium___1ocDap Text__heading_2___sUlNJP Heading__base___1dDlXY Heading__heading_2___3f_bIW'})
              headline = head.string
              date = soup.find('span', attrs = {'class': 'DateLine__date___12trWy'})
              final_date = date.string
              text = soup.find('div', attrs = {'class': 'ArticleBody__content___2gQno2 paywall-article'})
              paragraphs = text.find_all('p')
              text_final = ""
              for paragraph in paragraphs:
                  text_final = text_final + paragraph.getText()
          except:
              try:
                  head = soup.find('h1', attrs = {'class': 'Headline-headline-2FXIq Headline-black-OogpV ArticleHeader-headline-NlAqj'})
                  headline = head.string
                  date = json.loads(soup.select_one('[type="application/ld+json"]').contents[0])
                  final_date = date["datePublished"]
                  text = soup.find_all('p', attrs = {'class': 'Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x'})
                  text_final = ""
                  for paragraph in text:
                    text_final = text_final + paragraph.getText()
              except:
                  pass 
        except:
          pass
      
        locals()["df2" + str(company_name)] = pd.DataFrame({"Date":[final_date], "Headline":[headline], "Text":[text_final]})
        locals()["df" + str(company_name)] = locals()["df" + str(company_name)].append(locals()["df2" + str(company_name)])
      
      locals()["df" + str(company_name)].to_csv('{0}.csv'.format(company_name))
      # ["df" + str(company_name)].to_csv('{0}.csv'.format(company_name))
      # files.download('{0}.csv'.format(company_name))
      text = '{0}.csv'.format(company_name)
      print(text)
      !cp '{0}.csv'.format(company_name) "gdrive/My Drive/Colab Notebooks/FYP/REUTERS"